* Indexing method for making dictionary keys - DONE
* each position is an object with index but also list showing cRDS - DONE
* CAN PASS CARDS TO MC SO NO NEED TO BACK CONVERT. JUST USED AS A KEY. LIST IS USED FOR SCORING AND MC - DONE
* Create full list of positions - DONE
* Function to convert index keys to simulation inputs - DONE
* MC simulation
* Exact analytical solution
* Algo for using simulation to solve for strategy
* Compare analytical and numerical solution
* For playing games, the int indexes may just work. Just for strategy making they wont as those impossible hands are tehcnically possible. 


In [1]:
#Example index
17000000001001
#Example_hand
[7,0,0,0,0,0,0,0,0,10,1]
import random as rn
import numpy as np
import pandas as pd

In [2]:
def cardsToIndex(player_hand, dealer_card, draw = None):
    index_scaling = [10**11, 10**10, 10**9, 10**8, 10**7, 10**6, 10**5, 10**4, 10**2, 10**0]
    index = 10000000000000 + dealer_card*10**12
    if draw != None:
        player_hand[10 - draw] += 1
    for card in range(10):
        index += index_scaling[card] * player_hand[card]
    return index

In [3]:
def indexToCards(index):#TODO
    index = list(str(index)[1:])
    dealer_hand = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    dealer_hand[9 - int(index[0])] += 1
    index = index[1:]
    return [int(x) for x in index[0:-4] + [index[-4] + index[-3]] + [index[-2] + index[-1]]], dealer_hand

In [4]:
def score(player_cards):
    score = player_cards[-1] * 11
    high_aces = player_cards[-1]
    for cardIndex in range(9):
        score += (10 - cardIndex) * player_cards[cardIndex]
    #Determine whether to use aces as 1 or 11
    while score > 21 and high_aces > 0:
        if high_aces > 0:
            score -= 10
            high_aces -= 1
    return score 

In [5]:
class Position:
    def __init__(self, index = None, cards = None, dealer_card = None, draw = None, decks = 1):
        if index != None:
            self.index = index
            self.player_hand, self.dealer_hand = indexToCards(self.index)
        if index == None:
            self.player_hand = cards
            self.dealer_hand = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
            self.dealer_hand[10 - dealer_card] += 1
            self.index = cardsToIndex(self.player_hand, dealer_card, draw)

        self.dealer_face = dealer_card
        self.score = score(self.player_hand)
        self.bust = True if self.score > 21 else False
        self.decks = decks

        self.cards_in_play = [x + y for x, y in zip(self.player_hand, self.dealer_hand)]

        self.valid = True if all(x < 4*self.decks + 1  for x in self.cards_in_play[1:]) and self.cards_in_play[0] < 16*self.decks + 1 else False

    def add_stick_probability(self, stick_probability):
        self.stick_probability = stick_probability

In [6]:
def getPositions(decks = 1):
    Next = {10000000000000 : Position(index = 10000000000000, dealer_card = 1)}
    Master = {}
    runs = 0
    while len(Next) != 0:
        Current = Next
        Next = {}
        for position in Current.values():
            for drawn in range(1, 11):
                gate = True
                for dealer_card in range(1, 11):
                    temp = position.player_hand.copy()
                    current_position = Position(cards = temp, draw = drawn, dealer_card = dealer_card, decks = decks)
                    if not current_position.bust and current_position.valid:
                        if gate:
                            Next[current_position.index] = current_position
                            gate = False
                        if runs > 0:
                            Master[current_position.index] = current_position
        runs += 1
    return Master

In [7]:
Master = getPositions()

#### mc

# interesting
Yo umay be able to start simulating games for stick probs but just do all dealer possibilities, then you don't need to od te win check each time

In [8]:
def simulateGames(position, number_games = 1000):
    player_wins, dealer_wins, ties = 0, 0, 0
    dealer_card = position.dealer_face
    player_score = position.score
    #new
    draw_weights = np.array([x - y for x, y in zip([card*position.decks for card in [16, 4, 4, 4, 4, 4, 4, 4, 4, 4]], position.cards_in_play)])
    base_weights = []
    for i in range(10):
        base_weights += [10 - i]*draw_weights[i]

    base_dealer_hand = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    base_dealer_hand[10 - dealer_card] += 1
#---------------------------------------------------------------------
    while number_games > 0:
        temp_weights = base_weights.copy()
        dealer_hand = base_dealer_hand.copy()
        dealer_score = 0
        while dealer_score < 17:
            #card = np.random.choice([10, 9, 8, 7, 6, 5, 4, 3, 2, 1], 1, p=temp_weights/sum(temp_weights))[0]
            card = temp_weights.pop(rn.randint(0, len(temp_weights) - 1))
            dealer_hand[10 - card] += 1
            temp_weights[10 - card] -= 1
            dealer_score = score(dealer_hand)
#---------------------------------------------------------------------
        if player_score < 22:
            if player_score > dealer_score:
                player_wins += 1
            if player_score < dealer_score and dealer_score < 22:
                dealer_wins +=1
            if dealer_score > 21:
                player_wins +=1
            if player_score == dealer_score:
                ties += 1
        if player_score > 21:
            dealer_wins += 1

        number_games -= 1 
    return (player_wins - dealer_wins)/(player_wins + dealer_wins + ties)

In [9]:
simulateGames(position = Master[11000000000002], number_games = 250000)

-0.776616

In [19]:
for position in Master.values():
    position.add_stick_probability(simulateGames(position = position, number_games = 1000))

In [20]:
vars(Master[11000000000002])

{'player_hand': [0, 0, 0, 0, 0, 0, 0, 0, 0, 2],
 'dealer_hand': [0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
 'index': 11000000000002,
 'dealer_face': 1,
 'score': 12,
 'bust': False,
 'decks': 1,
 'cards_in_play': [0, 0, 0, 0, 0, 0, 0, 0, 0, 3],
 'valid': True,
 'stick_probability': -0.768}

In [28]:
simulateGames(position = Master[11000000000002], number_games = 250000)

-0.7798

In [47]:
import asyncio

async def main():
    async def one_iteration():
        
    coros = [one_iteration() for _ in range(2)]
    await asyncio.gather(*coros)

In [52]:
results = await asyncio.gather(simulateGames(position = Master[11000000000002], number_games = 250000), )

TypeError: An asyncio.Future, a coroutine or an awaitable is required

In [54]:
async def request():
    return simulateGames(position = Master[11000000000002], number_games = 250000)

if __name__ == '__main__':
    loop=asyncio.get_event_loop()
    loop.run_until_complete(request())

RuntimeError: This event loop is already running